In [5]:
# Bringing in the data

import pandas as pd

# Read the CSV file into a DataFrame
df_mm = pd.read_csv("C:/Users/hagen/Downloads/NBA DATA/Teams/Transformed Data/mm_data.csv")

df_p = pd.read_csv("C:/Users/hagen/Downloads/NBA DATA/Teams/Transformed Data/p_data.csv")

In [69]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

def SVM_top5(df, selected_features, C):
    """
    Custom cross-validation for predicting NBA champion using XGBoostRanker,
    with group information to rank the teams by their predicted level (score).
    """
    results = []

    # Iterate over the years you want to predict (2014 to 2023)
    for test_year in range(2014, 2024):
        
        # Training data: the previous 5 years of data (e.g., 2009-2013 for 2014 prediction)
        train_data = df[df['Year'].isin(range(test_year - 5, test_year))]
        
        # Test data: current year
        test_data = df[df['Year'] == test_year]
        
        # Prepare features and target for training and testing
        X_train = train_data[selected_features]
        y_train = train_data['Champion']
        
        X_test = test_data[selected_features]
        y_test = test_data['Champion']

        # Initialize the SVC model with probability=True
        model = SVC(probability=True, kernel='sigmoid', C=C, class_weight='balanced')

        # Train the model using SVM
        model.fit(X_train, y_train)
        
        # Make predictions on the test data
        y_pred = model.predict_proba(X_test)

        # Rank teams based on predicted scores
        ranked_teams = test_data[['Team']].copy()
        ranked_teams['Score'] = y_pred[:, 1]
        ranked_teams = ranked_teams.sort_values(by='Score', ascending=False)
        
        # Get the actual champion team (level 4) from the test data
        actual_champion = test_data[test_data['Levels'] == 4]['Team'].values[0]

        # Get top 5 teams and their scores
        top_teams = ranked_teams.head(5)
        top_teams_list = top_teams.apply(lambda row: f"{row['Team']}: {row['Score']:.4f}", axis=1).tolist()
        
        # Store results
        results.append([test_year, actual_champion] + top_teams_list)
    
    # Create dataframe with results
    columns = ['Year', 'Actual Champion', 'First Pick', 'Second Pick', 'Third Pick', 'Fourth Pick', 'Fifth Pick']
    results_df = pd.DataFrame(results, columns=columns)
    results_df = results_df.sort_values(by="Year", ascending=False)

    # Compute accuracy
    accuracy = (results_df['Actual Champion'] == results_df['First Pick'].str.split(': ').str[0]).mean()
    print(f"Model Accuracy: {accuracy * 100:.2f}%")
    
    return results_df

In [81]:
selected_features = ['W', 'PW', 'SRS', 'NRtg', 'MOV', 'eFG%', 'FG%', 'TS%']

SVM_top5(df_mm, selected_features, C=.5)

Model Accuracy: 50.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0729,Oklahoma City Thunder: 0.0647,Indiana Pacers: 0.0563,Denver Nuggets: 0.0532,Minnesota Timberwolves: 0.0528
8,2022,Denver Nuggets,Denver Nuggets: 0.0738,Philadelphia 76ers: 0.0692,Boston Celtics: 0.0682,Sacramento Kings: 0.0668,Cleveland Cavaliers: 0.0614
7,2021,Golden State Warriors,Phoenix Suns: 0.1111,Utah Jazz: 0.0935,Denver Nuggets: 0.0881,Golden State Warriors: 0.0877,Boston Celtics: 0.0824
6,2020,Milwaukee Bucks,Brooklyn Nets: 0.1159,Phoenix Suns: 0.1097,Utah Jazz: 0.1023,Los Angeles Clippers: 0.1014,Milwaukee Bucks: 0.1003
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.1018,Los Angeles Lakers: 0.0836,Utah Jazz: 0.0706,Los Angeles Clippers: 0.0695,Miami Heat: 0.0694
4,2018,Toronto Raptors,Golden State Warriors: 0.1065,Milwaukee Bucks: 0.0941,Toronto Raptors: 0.0801,Utah Jazz: 0.0637,Houston Rockets: 0.0589
3,2017,Golden State Warriors,Golden State Warriors: 0.0720,Houston Rockets: 0.0603,Toronto Raptors: 0.0540,Cleveland Cavaliers: 0.0428,Philadelphia 76ers: 0.0426
2,2016,Golden State Warriors,Golden State Warriors: 0.0382,Houston Rockets: 0.0276,San Antonio Spurs: 0.0270,Los Angeles Clippers: 0.0264,Cleveland Cavaliers: 0.0257
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.0263,San Antonio Spurs: 0.0226,Oklahoma City Thunder: 0.0192,Cleveland Cavaliers: 0.0163,Los Angeles Clippers: 0.0152
0,2014,Golden State Warriors,Golden State Warriors: 0.0300,Los Angeles Clippers: 0.0275,Atlanta Hawks: 0.0263,San Antonio Spurs: 0.0258,Cleveland Cavaliers: 0.0240


In [85]:
selected_features = ['W', 'PW', 'SRS', 'NRtg', 'MOV', 'eFG%', 'FG%', 'TS%']

SVM_top5(df_p, selected_features, C=2)

Model Accuracy: 40.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0735,Oklahoma City Thunder: 0.0717,Denver Nuggets: 0.0538,Minnesota Timberwolves: 0.0491,Indiana Pacers: 0.0431
8,2022,Denver Nuggets,Philadelphia 76ers: 0.0899,Denver Nuggets: 0.0883,Boston Celtics: 0.0832,Cleveland Cavaliers: 0.0782,Sacramento Kings: 0.0749
7,2021,Golden State Warriors,Phoenix Suns: 0.0537,Utah Jazz: 0.0443,Golden State Warriors: 0.0411,Miami Heat: 0.0342,Denver Nuggets: 0.0320
6,2020,Milwaukee Bucks,Phoenix Suns: 0.0841,Los Angeles Clippers: 0.0814,Brooklyn Nets: 0.0786,Milwaukee Bucks: 0.0770,Utah Jazz: 0.0718
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.0870,Los Angeles Lakers: 0.0668,Los Angeles Clippers: 0.0589,Golden State Warriors: 0.0571,Miami Heat: 0.0564
4,2018,Toronto Raptors,Milwaukee Bucks: 0.0612,Golden State Warriors: 0.0612,Toronto Raptors: 0.0569,Utah Jazz: 0.0437,Houston Rockets: 0.0358
3,2017,Golden State Warriors,Golden State Warriors: 0.0727,Toronto Raptors: 0.0632,Houston Rockets: 0.0625,Philadelphia 76ers: 0.0533,New Orleans Pelicans: 0.0417
2,2016,Golden State Warriors,Golden State Warriors: 0.0757,Los Angeles Clippers: 0.0616,Houston Rockets: 0.0607,San Antonio Spurs: 0.0566,Cleveland Cavaliers: 0.0558
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.0716,San Antonio Spurs: 0.0690,Oklahoma City Thunder: 0.0645,Cleveland Cavaliers: 0.0614,Los Angeles Clippers: 0.0592
0,2014,Golden State Warriors,Golden State Warriors: 0.1256,Los Angeles Clippers: 0.1179,Atlanta Hawks: 0.1100,San Antonio Spurs: 0.1070,Cleveland Cavaliers: 0.0941
